# Playing with Color
stough 202-

In this activity we'll use what we've learned about color to try and **enhance** or even **compress** them without too much perceptual loss. The other notebooks in this folder should be useful here, particularly [color_intro](./color_intro.ipynb), [color_HSV](./color_HSV.ipynb), and [color_YCbCr](./color_YCbCr.ipynb) 

## Imports
You can borrow the imports from any of the other notebooks in this directory.

In [5]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color
from ipywidgets import VBox, HBox, FloatSlider
from copy import deepcopy

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

import matrix_utils
from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_hists,
                       lab_uniform)

## 1. Enhancement: Improving the Color of an Image
Take a look at  `../dip_pics/sf.jpg` and try the `vis_hists` function on it to the distributions of the R,G,B channels. Of course you can try this out with any image you like of course.

In [6]:
I = plt.imread('../dip_pics/sf.jpg')

In [7]:
vis_hists(I)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

To me it looks like the clouds shrouding the [Golden Gate Bridge](https://en.wikipedia.org/wiki/Golden_Gate_Bridge) are too dark, not blue enough really for my tastes. You can see this in the relative color distributions as well, with the blue component maxing out below the midpoint of the intensity range. 

Try adding some blue to the image to really make those clouds pop better. **Be careful** to use the `arr_info` to understand both the `dtype` and understood range of your data. 

In [8]:
matrix_utils.arr_info(I)

((947, 1421, 3), dtype('uint8'), 0, 255)

In [9]:
bChannel = deepcopy(I)
bChannel[:,:,0]=0

fig, ax = plt.subplots(ncols=2)
ax[0].imshow(I)
ax[0].set_title("Original")

ax[1].imshow(bChannel)
ax[1].set_title("Blue Channel")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2. Ehancement: Improving Contrast through Saturation of Color
Take a look at `../dip_pics/world_overexposed.jpg`. Here we have a very overexposed image. (Again, find any overexposed image you like). What this means generally is that we have a lot of what should be color/Hue showing up as mostly white; as in, it's been mixed with too much white. See the [color_HSV](./color_HSV.ipynb) demo titled "Viewing the planes of the HSV space" and see what happens as saturation $S_{HSV}$ approaches zero. 

Try converting the image to HSV and then scaling the saturation component to improve the color purity of all those washed-out pixels. Then convert back to RGB and see if it's improved. **Remember**, from the [skimage documentation](https://scikit-image.org/docs/dev/api/skimage.color.html#skimage.color.rgb2hsv), that the hsv channels are all in $[0,1]$, so when you do the scaling you might need to `np.clip`. 

In [8]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color
from ipywidgets import VBox, HBox, FloatSlider
from copy import deepcopy

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import arr_info
from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_hists,
                       lab_uniform)

In [10]:
I = plt.imread('../dip_pics/world_overexposed.jpg')
vis_hists(I)
arr_info(I)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

((960, 1280, 3), dtype('uint8'), 31, 255)

In [11]:
In = I - I.min()
In = In/In.max()
In = (255*In).astype(np.uint8)
vis_hists(In)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
f, ax = plt.subplots(1,2, figsize=(8,3), sharex=True, sharey=True)
ax[0].imshow(I)
ax[1].imshow(In)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
HSV = color.rgb2hsv(I)

H, S, V = np.split(HSV, indices_or_sections=3, axis=-1)

In [13]:
arr_info(S)
plt.figure()
plt.imshow(S, cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
#S = newvalue*np.ones_like(V)
S2 = S.copy()/S.max()
S2[S2 > .2] = S2[S2 > .2] +.2
S2 = S2.clip(0, 1)

In [15]:
arr_info(S2)

((960, 1280, 1), dtype('float64'), 0.0, 1.0)

In [16]:
HSV = np.dstack((H,S2,V))
I_change = color.hsv2rgb(HSV)

In [17]:
f, ax = plt.subplots(1,2, figsize=(8,3), sharex=True, sharey=True)
ax[0].imshow(I)
ax[1].imshow(I_change)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
RGB = plt.imread('../dip_pics/world_overexposed.jpg')
HSV = color.rgb2hsv(RGB)

In [13]:
def changeSat(newSat):
    global HSV
    change = newSat - HSV[...,0].mean()
    
    IC = HSV.copy()
    IC[...,0] = np.fmod(IC[...,0]+change, 1)
    
    return np.clip(IC, 0, 1)

In [14]:
plt.ioff()
# plt.clf()

sat_mean = HSV[...,0].mean()

# A slider to set the saturation.
sat_slider = FloatSlider(
    orientation='horizontal',
    value=sat_mean,
    min=0.00,
    max=1.00,
    step=0.01,
    description='Saturation'
)

# Setting up the figure.
fig_args = {'num':' ', 'frameon':True, 'sharex':True, 'sharey':True}
fig, ax = plt.subplots(1,2, figsize=(8,3), **fig_args)

ih = changeSat(sat_mean)
irgb = color.hsv2rgb(ih)

odisp = ax[0].imshow(RGB)
rdisp = ax[1].imshow(irgb)

ax[0].set_title(f'Original ({sat_mean:.03f})')
rtext = ax[1].set_title(f'Saturation: {sat_mean:.03f}')

def update_image(change):
    global odisp, rdisp, HSV, rtext, ih
    
    I_rgb = color.hsv2rgb(changeSat(change.new))
    rdisp.set_array(I_rgb)
    rtext.set_text(f'Saturation: {change.new:.03f}')
    fig.canvas.draw()
    fig.canvas.flush_events()

sat_slider.observe(update_image, names='value')

plt.tight_layout()

VBox([sat_slider, fig.canvas])


/usr/remote/anaconda-3.7-deeplearn/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: tight_layout not applied: number of columns in subplot specifications must be multiples of one another.


In [21]:
## 3. Compression: Reducing the Bit-Depth of an Image to Save Space
In a prior activity we reduced the bit-depth of the RGB color channels of an image and saw how that *quantization* affected the image quality, creating sharp edges of color change in what would be smoothly graded areas. So one of the limits of potentially compressing images relates to the minimum bit-depth that can maintain perceptual fidelity.

But in this module's materials, we've seen that some color spaces (HSV, YCbCr, and L*a*b*) include some dimension that captures the brightness/luminance/intensity representation of an image separately from the "color" aspect. [YCbCr](./color_YCbCr.ipynb) in particular is used a lot in compression for precisely this reason.

In the below cells, try considering an image like `../dip_pics/bellagio.jpg` or `../dip_pics/mountainSpring.jpg` for example. 

- See what happens with you reduce the bit-depth to 4-4-4 in RGB. 
- Then try reducing the bit-depth to 6-3-3 but **in the YCbCr** space. Here we're saving the same number of bits per pixel, but differentially quantizing (and therefore compressing) the **Luminance** and color components.

It may be useful here to define a function that quantizes a 2D `ndarray` instead of a whole image at once. Luckily, it's likely the same quantization function you've already written may suffice. I include a useable version here just to help.  

**Be careful**: [`skimage.color.ycbcr2rgb`](https://scikit-image.org/docs/dev/api/skimage.color.html#skimage.color.ycbcr2rgb) expects 'float64' data type. 

SyntaxError: invalid syntax (<ipython-input-21-eb0116c8719e>, line 2)

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import skimage.color as color
from ipywidgets import VBox, HBox, FloatSlider
from copy import deepcopy

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import arr_info
from vis_utils import (vis_rgb_cube,
                       vis_hsv_cube,
                       vis_hists,
                       lab_uniform)

def quantize_bitdepth(C, bitdepth=8):
    """
    quantize_bitdepth(C, bitdepth=8): return a copy of the array C where the elements are
    limited to 2**bitdepth unique values. Tries to handle a float image just in case.
    """
    assert bitdepth <= 8, f'quantize_bitdepth error: expects bitdepth <= 8, got {bitdepth}'
    
    shft_amt = 8 - bitdepth
    
    J = np.uint8(C)
    if C.dtype == 'float':
        J = np.uint8(256 * (J/J.max()))
        
    return np.uint8((J >> shft_amt)*(255/(2**bitdepth-1)))


B = plt.imread('../dip_pics/bellagio.jpg')
YCb = color.rgb2ycbcr(B)

fig_args = {'num':' ', 'frameon':True, 'sharex':True, 'sharey':True}
fig, ax = plt.subplots(1,3, figsize=(8,3), **fig_args)

QB = quantize_bitdepth(B, 1)
Qycb = quantize_bitdepth(YCb, 4)

orig = ax[0].imshow(B)
rgbnew = ax[1].imshow(QB)
ycbnew = ax[2].imshow(color.ycbcr2rgb(Qycb.astype('float64')))

#ycb = ax[2].imshow(Qybc)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
